## Stadtentwicklung

Angenommen, Sofia steht vor einem lokalen Haushaltsdefizit und der Stadtrat überlegt, wie die Grundsteuereinnahmen durch die Sanierung städtischer Grundstücke erhöht werden können. Das Projekt besteht aus zwei Teilen: der Beseitigung vernachlässigter und beschädigter Gebäude und dem Bau neuer Wohnungen.

1. Die Gemeinde besitzt derzeit 300 baufällige Gebäude, die abgerissen werden könnten. Jedes dieser Gebäude macht 1,000 Quadratmeter frei und kostet 3,000 EUR pro Gebäude. 15 Prozent der befreiten Fläche sind für Straßen, Gehwege und Freiflächen vorgesehen.
2. Auf den geräumten Grundstücken kann die Gemeinde vier Arten von neuen Wohngebäuden errichten: Einfamilienhäuser (300 Quadratmeter), Zweifamilienhäuser (500 Quadratmeter), Dreifamilienhäuser (700 Quadratmeter) und Vierfamilienhäuser (900 Quadratmeter). Die geschätzten Steuereinnahmen belaufen sich auf EUR 1,000, EUR 1.700,  EUR 2,400 bzw.  EUR 2,800 pro Jahr.
3. Mindestens 20 Prozent der Neubauten müssen Einfamilienhäuser sein, Zweifamilienhäuser müssen mindestens 20 Prozent ausmachen, und Drei- und Vierfamilienhäuser müssen (zusammen) mindestens ein Viertel aller Neubauten ausmachen.
4. Die Baukosten für neue Häuser betragen 50,000 EUR, 70,000 EUR, 130,000 EUR bzw. 160,000 EUR.
5. Die Gemeinde beabsichtigt, das Projekt durch ein Bankdarlehen zu finanzieren, das 15 Millionen Pfund nicht überschreiten darf.

Wie viele Häuser jedes Typs sollte die Gemeinde bauen, um die höchstmöglichen Steuereinnahmen zu erzielen?

Die hausbezogenen Daten aus der Aufgabenstellung sind in dem Datensatz `homes` enthalten. Die Spalten sind wie folgt:

- `cost`: Die Baukosten für ein Haus dieses Typs.
- `tax`: Die geschätzten Steuereinnahmen pro Jahr für ein Haus dieses Typs.
- `area`: Die Fläche, die ein Haus dieses Typs einnimmt.


In [2]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

homes = pd.DataFrame({
    'type': ['single', 'double', 'triple', 'quad'],
    'cost': [50000, 70000, 130000, 160000],
    "tax": [1000, 1700, 2400, 2800],
    'area': [300, 500, 700, 900],
}).set_index('type')

homes

,cost,tax,area
size,,,
single,50000,1000,300
double,70000,1700,500
triple,130000,2400,700
quad,160000,2800,900


In [ ]:
model1 = gp.Model('Beispiel')

x = model1.addVars(homes.index, vtype=GRB.INTEGER, name='x')
model1.update()

x

{'single': <gurobi.Var x[single]>,
 'double': <gurobi.Var x[double]>,
 'triple': <gurobi.Var x[triple]>,
 'quad': <gurobi.Var x[quad]>}

In [46]:
x['single']

<gurobi.Var x[single]>

In [47]:
x['quad']

<gurobi.Var x[quad]>

In [ ]:
x['single'] + x['quad']

<gurobi.LinExpr: x[single] + x[quad]>

In [52]:
x.sum()

<gurobi.LinExpr: x[single] + x[double] + x[triple] + x[quad]>

In [50]:
homes['area'].to_dict()

{'single': 300, 'double': 500, 'triple': 700, 'quad': 900}

In [53]:
x.prod(homes['area'].to_dict()) <= 2000

<gurobi.TempConstr: 300.0 x[single] + 500.0 x[double] + 700.0 x[triple] + 900.0 x[quad] <= 2000>

In [41]:
x['single']

<gurobi.Var *Awaiting Model Update*>

In [ ]:
m = gp.Model('homes')

x = m.addVars(homes.index, vtype=GRB.INTEGER, name='x')
x_d = m.addVar(vtype=GRB.INTEGER, name='demolished')

# Die Zielfunktion

# m.setObjective(..., GRB.MAXIMIZE)

# Die Einschränkungen

## Kosten
# m.addConstr(x.prod(homes['cost'].to_dict()) + ... <= 15e6, 'costs')

## Fläche
# m.addConstr(... <= ..., 'area')

## Anzahl der Einfamilienhäuser
m.addConstr(x['single'] >= 0.2 * x.sum(), 'single homes')

## Anzahl der Zweifamilienhäuser
# m.addConstr(..., 'double homes')

## Anzahl der Drei- und Vierfamilienhäuser
# m.addConstr(..., 'triple and quad homes')

# Die Lösung

# m.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 5 125U, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 14 logical processors, using up to 14 threads

Optimize a model with 5 rows, 5 columns and 22 nonzeros
Model fingerprint: 0xfda95f49
Coefficient statistics:
  Matrix range     [2e-01, 2e+05]
  Objective range  [1e+03, 3e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+07, 2e+07]
Presolve removed 2 rows and 1 columns
Presolve time: 0.01s
Presolved: 3 rows, 4 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.4728299e+05   6.956791e+00   0.000000e+00      0s
       1    3.1533900e+05   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds (0.00 work units)
Optimal objective  3.153389994e+05


In [ ]:
# m.write('homes.lp')

# with open('homes.lp', 'r') as f:
#     print(f.read())

\ Model homes
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  1000 x[single] + 1700 x[double] + 2400 x[triple] + 2800 x[quad]
Subject To
 costs: 50000 x[single] + 70000 x[double] + 130000 x[triple]
   + 160000 x[quad] + 3000 x_d <= 1.5e+07
 area: 300 x[single] + 500 x[double] + 700 x[triple] + 900 x[quad]
   - 999.85 x_d <= 0
 single_homes: 0.8 x[single] - 0.2 x[double] - 0.2 x[triple] - 0.2 x[quad]
   >= 0
 double_homes: - 0.2 x[single] + 0.8 x[double] - 0.2 x[triple]
   - 0.2 x[quad] >= 0
 triple_and_quad_homes: - 0.25 x[single] - 0.25 x[double] + 0.75 x[triple]
   + 0.75 x[quad] = 0
Bounds
Generals
 x[single] x[double] x[triple] x[quad] x_d
End



In [ ]:
# Die Lösung als DataFrame

# vars_df = pd.DataFrame(
#     [(v.varName, v.x) for v in m.getVars()],
#     columns=['variable', 'value']
# )

# vars_df

,variable,value
0,x[single],36.0
1,x[double],99.0
2,x[triple],41.0
3,x[quad],4.0
4,x_d,93.0


In [ ]:
# Die Einschränkungen as DataFrame

# constrs_df = pd.DataFrame(
#     [(c.constrName, c.slack) for c in m.getConstrs()],
#     columns=['constraint', 'slack']
# )

# constrs_df

,constraint,slack
0,costs,0.000000
1,area,0.000000
2,single homes,0.000000
3,double homes,-63.613055
4,triple and quad homes,0.000000
